# Fleche Storage Backends

This notebook demonstrates the usage of the different storage backends available in `fleche`.
In `fleche`, a `Cache` is composed of two storage components:
1. **values**: Stores the actual results of the functions.
2. **calls**: Stores the metadata about the function calls (arguments, function name, etc.) and references to the stored values.

You can mix and match different storage backends for values and calls to suit your needs.

## Memory Storage

The `Memory` storage backend keeps all the cached data in memory. This is the simplest backend and is useful for testing or when you don't need to persist the cache beyond the current process.

In [1]:
from fleche import fleche, cache, Cache
from fleche.storage import Memory

# Using Memory for both values and calls
memory_cache = Cache(values=Memory({}), calls=Memory({}))

with cache(memory_cache):
    @fleche
    def add(a, b):
        print(f"Executing add({a}, {b})")
        return a + b
    
    print(f"Result 1: {add(2, 3)}")
    print(f"Result 2: {add(2, 3)}") # This should be cached
print('Cache keys in calls storage:', list(memory_cache.calls.list()))

LOGGING INFO: local config fleche.toml does not exist, trying global
LOGGING INFO: global config /home/jules/.fleche.toml does not exist
LOGGING INFO: local config fleche.toml does not exist, trying global
LOGGING INFO: global config /home/jules/.fleche.toml does not exist
Executing add(2, 3)
Result 1: 5
Result 2: 5
Cache keys in calls storage: ['dab010f5591977ad591e8a7bc578a25ba978f729f60654fb4eecd071b76cb29e']


## PickleFile Storage

The `PickleFile` storage backend serializes data using Python's standard `pickle` module and stores it in individual files. It requires a `root` directory.

In [2]:
import shutil
from pathlib import Path
from fleche.storage import PickleFile

shutil.rmtree('.pickle_cache', ignore_errors=True)
pickle_cache = Cache(
    values=PickleFile(root='.pickle_cache/values'),
    calls=PickleFile(root='.pickle_cache/calls')
)

with cache(pickle_cache):
    @fleche
    def greet(name):
        print(f"Executing greet({name})")
        return f"Hello, {name}!"
    
    print(greet("World"))
    print(greet("World"))

print("Files in calls storage:")
!ls .pickle_cache/calls

Executing greet(World)
Hello, World!
Hello, World!
Files in calls storage:
03b6cfa8cac27e4d9fdb048035e96b2bc92aecab5600ac14cb69537a96af1074


## CloudpickleFile Storage

The `CloudpickleFile` storage backend is similar to `PickleFile` but uses `cloudpickle` for serialization. `cloudpickle` can handle more complex Python objects, like lambdas or functions defined interactively, that standard `pickle` might struggle with.

In [3]:
from fleche.storage import CloudpickleFile

shutil.rmtree('.cloudpickle_cache', ignore_errors=True)
cp_cache = Cache(
    values=CloudpickleFile(root='.cloudpickle_cache/values'),
    calls=CloudpickleFile(root='.cloudpickle_cache/calls')
)

with cache(cp_cache):
    @fleche
    def mul(a, b):
        print(f"Executing mul({a}, {b})")
        return a * b
    
    print(f"Result: {mul(3, 4)}")
    print(f"Result: {mul(3, 4)}")

print("Files in values storage:")
!ls .cloudpickle_cache/values

Executing mul(3, 4)
Result: 12
Result: 12
Files in values storage:
056a34cfa7662755ef47b05e966a8cd152ad9cd663134c03b0a4e6f61349c7e5
65d52a82c5a72f12ca0499522dc9274a0e6822e1038630ba68f94400b3e4c98f
83ada2198553b88cb3d0882f7fca8c4e9531049b978df3e9e3b5d6301c6c0bfa


## BagOfHoldingH5File Storage

The `BagOfHoldingH5File` storage backend uses the `bagofholding` library to store data in HDF5 files. This is particularly efficient for large numerical arrays (e.g., NumPy arrays).

In [4]:
import numpy as np
from fleche.storage import BagOfHoldingH5File

shutil.rmtree('.boh_cache', ignore_errors=True)
boh_cache = Cache(
    values=BagOfHoldingH5File(root='.boh_cache/values'),
    calls=CloudpickleFile(root='.boh_cache/calls') # We can use Cloudpickle for calls
)

with cache(boh_cache):
    @fleche
    def make_array(n):
        print(f"Executing make_array({n})")
        return np.ones((n, n))

    print(f"Array sum: {make_array(5).sum()}")
    print(f"Array sum: {make_array(5).sum()}")

print("Files in H5 values storage:")
!ls .boh_cache/values

Executing make_array(5)
Array sum: 25.0
Array sum: 25.0
Files in H5 values storage:
5b07a837d91d67764109c11fb912c9b91b4e9d9d4c909fca542e5fbe39ddc244
959e3f19a52231949b7ef1569a103d887490c5b588eadb46c39d95cda7c28a90


## Sql Storage (Call Storage Only)

The `Sql` storage backend uses SQLAlchemy to store call metadata in a SQL database (like SQLite). It provides advanced querying capabilities but can only be used for the **calls** component of a `Cache`.

In [5]:
from fleche.storage import Sql
import os

if os.path.exists('calls.db'): os.remove('calls.db')

sql_cache = Cache(
    values=Memory({}),
    calls=Sql(url='sqlite:///calls.db')
)

with cache(sql_cache):
    @fleche
    def power(a, b):
        print(f"Executing power({a}, {b})")
        return a ** b

    power(2, 8)
    power(2, 8)

print("Calls in SQL storage:")
print(list(sql_cache.calls.list()))

Executing power(2, 8)
Calls in SQL storage:
['f28dcfef076012d3901e0f4863f4be6102659575430cdf5d95af8190a93814da']


## Mix and Match

As seen in the `BagOfHoldingH5File` and `Sql` examples, you can mix different backends for values and calls. For instance, you might want to use `BagOfHoldingH5File` for large values but `Sql` for calls to enable efficient metadata querying.

In [6]:
mixed_cache = Cache(
    values=BagOfHoldingH5File(root='.mixed_cache/values'),
    calls=Sql(url='sqlite:///.mixed_cache/calls.db')
)

with cache(mixed_cache):
    @fleche
    def compute_heavy(x):
        return np.random.rand(x, x)
    
    compute_heavy(10)
    
print(f"Value storage: {type(mixed_cache.values).__name__}")
print(f"Call storage: {type(mixed_cache.calls).__name__}")

Value storage: BagOfHoldingH5File
Call storage: Sql


# Clean Up

Remove temporary cache directories.

In [7]:
!rm -rf .pickle_cache .cloudpickle_cache .boh_cache .mixed_cache calls.db